In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from glob import glob

## Reading the image:

In [ ]:
# Uncomment if you are in Google Colab:

# !pip install wget
# import wget, os
# url0 = "https://github.com/IvanSol/ml-course/raw/refs/heads/master/Class05_classic_CV/data/"
# urls = [url0 + fn for fn in ['cat.png', 'day00.png', 'day01.jpg', 'day02.jpg', 'day03.jpg', 'day04.jpg', 'night00.jpg', 'night01.jpg', 'night02.png', 'night03.jpg', 'night04.png', 'board.png']]
# os.makedirs('data', exist_ok=True)   
# for url in urls:
#   wget.download(url, out=os.path.join('data', url.split('/')[-1]))

In [ ]:
cat = cv2.imread('data/cat.png')
plt.imshow(cat)

### Colors seem to be incorrect. This happens because opencv by default stores images as BGR, not RGB.
There is a special function for converting the colors:

In [ ]:
cat_rgb = cv2.cvtColor(cat, cv2.COLOR_BGR2RGB)
plt.imshow(cat_rgb)

Image is stored as 3-d array: (H, W, C), C -- channels. For RGB image there are 3 channels: R, G, B.

In [ ]:
cat.shape

So, actually, to turn BGR image to RGB we may just inverse order of channels:

In [ ]:
cat_rgb2 = cat[:,:,::-1]
plt.imshow(cat_rgb2)

Classical computer vision algorithms often work with grayscale image, so, let us convert image to grayscale:

In [ ]:
cat_gray = cv2.cvtColor(cat, cv2.COLOR_BGR2GRAY)
print(cat_gray.shape)
plt.imshow(cat_gray, cmap='gray')

Sometimes we may select some parts of the image according to some criteria, then selected parts may be stored in binary image which contains 0 or 1:

In [ ]:
cat_bin = cat_gray < 10
plt.imshow(cat_bin, cmap='gray')

In [ ]:
def get_kernel(n):
    # Let's define our kernel size
    kernel = np.zeros((n,n), np.uint8)
    kernel = cv2.circle(kernel, (n//2, n//2), n//2, 1, -1)
    return kernel    

In [ ]:
# Now we erode
cat_2 = cv2.erode(cat_bin.astype(np.uint8), get_kernel(3))
cat_2 = cv2.dilate(cat_2.astype(np.uint8), get_kernel(35))
plt.imshow(cat_2)

In [ ]:
info, component_map = cv2.connectedComponents(cat_2.astype(np.uint8))
print(info)
plt.imshow(component_map)

## Task: Write a function to calculate how many red circles are there on the image (3 points).

In [ ]:
W, H = 1000, 1000
img = np.ones((H, W, 3)) * 255
colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255)]
for i in range(100):
    ok = False
    while not ok:
        x, y = np.random.randint(0, 1000, 2)
        min_r = min([50, x, y, W - x, H - y])
        if min_r <= 10:
            continue
        r = np.random.randint(10, min_r)
        c = colors[np.random.randint(3)]
        m = img[y - r : y + r + 1, x - r : x + r + 1]
        mask = np.ones_like(m) * 255
        cv2.circle(mask, (r, r), r, 0, -1)
        if (m[mask > 0] == 255).all():
            ok = True
            img = cv2.circle(img, (x, y), r, c, -1)
#plt.imshow(img)

In [ ]:
plt.imshow(img)

In [ ]:
def count_red_circles(img):
    pass # YOUR CODE HERE

In [ ]:
img = cv2.imread('data/board.png', cv2.IMREAD_GRAYSCALE)
plt.imshow(img, cmap='gray')

In [ ]:
gray = abs(img[1:].astype(float) - img[:-1].astype(float))
gray = cv2.dilate(gray.astype(np.uint8), get_kernel(5))
plt.imshow(gray>0, cmap='gray')

In [ ]:
# It contains code with geometrical functions to calculate distance between lines:
from remove_similar_lines import remove_similar_lines

def detect_lines(gray, threshold=10, min_line_length=1000, max_line_gap=100):
    """
    Detects lines on the image using Hough transform.
    """
    blurred = (cv2.GaussianBlur(gray, (3, 3), 0) > 0).astype(np.uint8)
    lines = cv2.HoughLinesP(blurred, 
                            rho=1, 
                            theta=np.pi/180, 
                            threshold=100,
                            minLineLength=min_line_length,
                            maxLineGap=max_line_gap)
    
    image_with_lines = np.zeros_like(gray)
    # This function calculates paiwise distances between lines and removes ones which are close to each other.
    lines = remove_similar_lines([l[0] for l in lines])
    
    # Draw found lines:
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line
            cv2.line(image_with_lines, (x1, y1), (x2, y2), 255, 2)
    
    return image_with_lines, lines

In [ ]:
grad_y = abs(img[1:] - img[:-1])
plt.imshow(grad_y)
plt.show()

In [ ]:
result_image, lines = detect_lines(grad_y)
print(f"Lines found: {len(lines) if lines is not None else 0}")
plt.imshow(result_image)

## Task: Color all black cells in even rows with green (5 points).

In [ ]:
def recolor_even_rows(img):
    pass # YOUR CODE HERE

## Image classification

In [ ]:
day_imgs = []
night_imgs = []
for fn in glob('data/day*'):
    day_imgs.append(cv2.imread(fn, cv2.IMREAD_GRAYSCALE))
for fn in glob('data/night*'):
    night_imgs.append(cv2.imread(fn, cv2.IMREAD_GRAYSCALE))

In [ ]:
plt.imshow(day_imgs[0], cmap='gray')
plt.show()
plt.imshow(night_imgs[0], cmap='gray')
plt.show()

#### Here we may find features to be used for classification.